In [1]:
from digital_hospitals.bim import models
import networkx as ntx
from datetime import datetime
import pytz

from pymongo import MongoClient
from digital_hospitals.common import MONGODB_PORT, MONGODB_USER, MONGODB_PASSWORD
client = MongoClient("localhost", MONGODB_PORT, username=MONGODB_USER, password=MONGODB_PASSWORD, timeoutMS=5000)

requested = datetime.now().timestamp()
requested

1717450915.51216

In [2]:
LONDON = pytz.timezone('Europe/London')
my_dt = datetime.fromtimestamp(requested, LONDON)
print(my_dt.__repr__())
print(my_dt.isoformat())

datetime.datetime(2024, 6, 3, 22, 41, 55, 512160, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)
2024-06-03T22:41:55.512160+01:00


In [3]:
db = client['test-db']
bim = db['bim']
bim_latest = db['bim-latest']

In [4]:
_id = bim.insert_one({
    'requested': requested,
    'status': 'Running'
}).inserted_id

In [5]:

model = models.BimModel.from_ifc('../secrets/histo.ifc')
door_list = model.doors.loc[model.doors.door_name.str.fullmatch(r'd\d+')].door_name.to_list()
extra_paths = [
    models.Path(path=['d10', 'd12'], duration_seconds=120., required_assets=['Lift']),
    models.Path(path=['d11', 'd13'], duration_seconds=45., required_assets=['Lift'])
]
g = models.logical_graph(
    model,
    door_list,
    extra_paths,
    models.DEFAULT_RUNNER_SPEED
)

data = ntx.node_link_data(g)
data

{'directed': False,
 'multigraph': False,
 'graph': {},
 'nodes': [{'id': 'd1'},
  {'id': 'd2'},
  {'id': 'd3'},
  {'id': 'd4'},
  {'id': 'd5'},
  {'id': 'd6'},
  {'id': 'd7'},
  {'id': 'd8'},
  {'id': 'd9'},
  {'id': 'd10'},
  {'id': 'd11'},
  {'id': 'd12'},
  {'id': 'd13'},
  {'id': 'd14'},
  {'id': 'd15'},
  {'id': 'd16'}],
 'links': [{'weight': 4.8570226039551585, 'source': 'd1', 'target': 'd2'},
  {'weight': 7.3570226039551585, 'source': 'd1', 'target': 'd3'},
  {'weight': 7.7736892706218255, 'source': 'd1', 'target': 'd4'},
  {'weight': 14.44035593728849, 'source': 'd1', 'target': 'd5'},
  {'weight': 14.44035593728849, 'source': 'd1', 'target': 'd6'},
  {'weight': 7.6725889843221236, 'source': 'd1', 'target': 'd7'},
  {'weight': 5.934433619633036, 'source': 'd1', 'target': 'd10'},
  {'weight': 3.1903559372884915, 'source': 'd2', 'target': 'd3'},
  {'weight': 3.2618446353109127, 'source': 'd2', 'target': 'd4'},
  {'weight': 10.273689270621826, 'source': 'd2', 'target': 'd5'},
  {'

In [6]:
bim.find_one_and_update(
    {'_id': _id},
    {'$set': {'status': 'OK', 'result': data}}
)

{'_id': ObjectId('665e38a3bb2575e94eb6e40e'),
 'requested': 1717450915.51216,
 'status': 'Running'}

In [7]:
item = bim.find_one({'_id': _id})
del item['_id']  # The _id field in MongoDB cannot be changed

In [8]:
old_item = bim_latest.find_one()
if old_item is None:
    print('bim-latest is empty')
    bim_latest.insert_one(item)
    print('Updated!')
elif old_item['requested'] >= item['requested']:
    print('New value is not newer than bim-latest; discarding')
else:
    bim_latest.find_one_and_replace({}, item)
    print('Updated!')


bim-latest is empty
Updated!


In [9]:
client.close()